In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-approval-prediction/credit_record.csv
/kaggle/input/credit-card-approval-prediction/application_record.csv
/kaggle/input/credit-record/credit_record.csv


# 1. Data Structure Analysis

In [2]:
# Reading both csv for further manipulations
application_df = pd.read_csv("/kaggle/input/credit-card-approval-prediction/application_record.csv")
credit_record_df = pd.read_csv("/kaggle/input/credit-card-approval-prediction/credit_record.csv")

# Number of clients
comparison = application_df[['ID']].drop_duplicates().merge(
    credit_record_df[['ID']].drop_duplicates(),
    on='ID',
    how='outer',
    indicator=True)

only_in_a = comparison[comparison['_merge'] == 'left_only']
only_in_b = comparison[comparison['_merge'] == 'right_only']
in_both = comparison[comparison['_merge'] == 'both']

print('Total application record:',len(application_df[['ID']].drop_duplicates()))
print('Just in application record:',len(only_in_a))
print('Just in credit record:',len(only_in_b))
print('In both:',len(in_both))

Total application record: 438510
Just in application record: 402053
Just in credit record: 9528
In both: 36457


In [3]:
#402,053 out of 438,557 records (91.7%) do not match across the two tables. 
#This can be an hypothesis of structural inconsistencies in the data pipeline or 
#historical availability, and indicates that any downstream analysis should be interpreted with caution until lineage validation is completed.
print('Percentage of data in both:',round(len(in_both)/len(application_df[['ID']].drop_duplicates()),3)*100,'%')

Percentage of data in both: 8.3 %


In [4]:
# When analyzing if any column have null values, it was found that almost 31% in OCCUPATION_TYPE is null
application_df.isnull().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

In [5]:
# Even with OCCUPATION_TYPE being null, DAYS_EMPLOYED have negative values (indicating that the client is still working) and a positive value '365243'
application_df[application_df['OCCUPATION_TYPE'].isnull()].head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0
8,5008813,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0
9,5008814,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,NaN,1.0


In [6]:
# Analyzing this DAYS_EMPLOYED == 365243, it was possible to see that only pensioners have this value, indicating they are not working
# Also, we have 164 pensioners with value on DAYS_EMPLOYED

just_365243 = application_df[application_df['DAYS_EMPLOYED']==365243]
print('Types of values in name income type:',just_365243['NAME_INCOME_TYPE'].unique())

just_pensioner = application_df[application_df['NAME_INCOME_TYPE']=='Pensioner']
print('Quantity of pensioners:',len(just_pensioner))
print('Quantity of pensioners with other values:',len(just_pensioner[just_pensioner['DAYS_EMPLOYED']!=365243]))


Types of values in name income type: ['Pensioner']
Quantity of pensioners: 75493
Quantity of pensioners with other values: 164


# 2. Pattern Investigation and Data Analysis

## 2a. Additional Variables for Deep Dive

In [7]:
# Creating AGE, MONTHS_EMPLOYED and YEARS_EMPLOYED
application_df['AGE'] = round((application_df['DAYS_BIRTH']*-1)/365)
application_df['MONTHS_EMPLOYED'] = round((application_df['DAYS_EMPLOYED']*-1)/30)
application_df['YEARS_EMPLOYED'] = round((application_df['DAYS_EMPLOYED']*-1)/365,1)

# Creating Cluster of months employed for further deep-dive
application_df['CLUSTER OF MONTHS EMPLOYED'] = np.where(
     application_df['MONTHS_EMPLOYED'].between(-99999, 0, inclusive='both'), 
    '0. Unemployed', np.where(
     application_df['MONTHS_EMPLOYED'].between(0, 6, inclusive='both'), 
    '1. Employed between 0 and 6 months', np.where(
     application_df['MONTHS_EMPLOYED'].between(7, 12, inclusive='both'), 
    '2. Employed between 7 months to 1 year', np.where(
     application_df['MONTHS_EMPLOYED'].between(13, 18, inclusive='both'), 
    '3. Employed between 1 year and 1 month to 1 year and half ', np.where(
     application_df['MONTHS_EMPLOYED'].between(19, 24, inclusive='both'), 
    '4. Employed between 1 year and 7 months to 2 years', np.where(
     application_df['MONTHS_EMPLOYED'].between(25, 36, inclusive='both'), 
    '5. Employed between 2 years and one month to 3 years', np.where(
     application_df['MONTHS_EMPLOYED'].between(37, 60, inclusive='both'), 
    '6. Employed between 3 years and one month to 5 years', np.where(
     application_df['MONTHS_EMPLOYED'].between(61, 84, inclusive='both'), 
    '7. Employed between 5 years and one month to 7 years',np.where(
     application_df['MONTHS_EMPLOYED'].between(85, 108, inclusive='both'), 
    '8. Employed between 7 years and one month to 9 years',
    '9. Employed more than 9 years')))))))))

In [8]:
# Importing the credit record for analyzing the last 12 months default
credit_df = pd.read_csv('/kaggle/input/credit-record/credit_record.csv',delimiter=';')

# Asserting how many clients on default this month
most_recent_month = credit_df[credit_df['MONTHS_BALANCE']==0]
print('Quantity of clients operating this month:',len(most_recent_month))
print('Quantity of clients with default this month:',len(most_recent_month[most_recent_month['STATUS'].isin(['1','2','3','4','5'])]))

Quantity of clients operating this month: 33856
Quantity of clients with default this month: 404


In [9]:
# Sorting decrescenting as 0 is the most recent date and -11 is the past 12 month
credit_df.sort_values(['ID','MONTHS_BALANCE'],
                      ascending=[True, False])

# Getting the last 12 months to create a variable for further deep dive
last_12 = credit_df[credit_df['MONTHS_BALANCE'].between(-11,0)]
print('='*100)
print('Quantity of clients operating the last 12 months:',len(last_12['ID'].unique()))
print('='*100)

# Considering that the default is accumulative, so if it has default90, it also have default 60 and 30.
# Status description: 
# C = Paid default
# X = No default at the moment
# 0 = 0 to 29 days of default
# 1 = 30 to 59 days of default
# 2 = 60 to 89 days of default
# 3 = 90 to 119 days of default
# 4 = 120 to 149 days of default
# 5 = bad debt or more than 150 days of default
flag_df = (last_12.assign(default30=last_12['STATUS'].isin(['1','2','3','4','5']))
           .assign(default60=last_12['STATUS'].isin(['2','3','4','5']))
           .assign(default90=last_12['STATUS'].isin(['3','4','5']))
           .assign(default120=last_12['STATUS'].isin(['4','5'])))



Quantity of clients operating the last 12 months: 38862


In [10]:
# unifying in a single row
defaults = ['default30', 'default60', 'default90','default120']

unify = (
    flag_df
    .groupby('ID', as_index=False)[defaults]
    .max()
)

unify.rename(columns=lambda c: f'{c}_last_12m' if c in defaults else c, inplace=True)

print('Quantity of clients with 30-days default in the last 12 months:',len(unify[unify['default30_last_12m']==True]))

Quantity of clients with 30-days default in the last 12 months: 2225


In [11]:
# Exporting for analyzing
flag_df.to_csv('credit_record_flagged.csv')
